In [1]:
#예상 투자 비용 계산기
 
import pandas as pd

# 양돈업 투자 비용 계산기 함수
def calculate_pig_farm_investment(
    num_pigs,
    cost_feed,
    cost_building,
    water_electricity_cost,
    disease_control_cost,
    vehicle_cost,
    baby_pig_cost,
    labor_cost,
    manure_disposal_cost,
    general_management_cost,
    land_cost,
    other_costs
):
    # 각 비용 항목 합산
    total_cost = (
        cost_feed + cost_building + water_electricity_cost +
        disease_control_cost + vehicle_cost + baby_pig_cost + labor_cost +
        manure_disposal_cost + general_management_cost + land_cost + other_costs
    ) * num_pigs

    return total_cost

# 사용자 입력으로 투자비용 계산
def main():
    num_pigs = int(input("사육할 돼지 마리수를 입력하세요: "))

    total_investment = calculate_pig_farm_investment(
        num_pigs=num_pigs,
        cost_feed=238468,                # 마리당 사료비
        cost_building=9644,              # 마리당 시설비
        water_electricity_cost=7533,     # 마리당 수도광열비
        disease_control_cost=10637,      # 마리당 방역치료비
        vehicle_cost=1798,               # 마리당 차량비
        baby_pig_cost=82075,             # 마리당 재료비
        labor_cost=25482,                # 마리당 노동비
        manure_disposal_cost=12405,      # 마리당 분뇨처리비
        general_management_cost=2155,    # 마리당 일반관리비
        land_cost=16779,                 # 마리당 토지비
        other_costs=16879                # 마리당 기타비용
    )

    print(f" 예상 총 투자 비용: {total_investment:,.0f}원 (총 {num_pigs}마리 기준)")

if __name__ == "__main__":
    main()


 예상 총 투자 비용: 66,969,090원 (총 158마리 기준)


In [2]:
#전국 평균 돼지 경락가격 (1kg당)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# 1. 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 2. 대상 페이지 열기
url = "https://www.ekapepia.com/supPrice/auction/price/pigNew.do?menuSn=65&boardInfoNo=&userGroupType=39&searchStartDate=2025-03-25&searchEndDate=2025-03-25&pageType=main&gubunType=#!"
driver.get(url)

# 3. 페이지 로딩 대기
time.sleep(3)

# 4. CSS 선택자로 요소 찾기
try:
    selector = "#content > div.sub-in > section:nth-child(10) > div > div.table-basic.ofx > table > tbody > tr:nth-child(7) > td:nth-child(19)"
    element = driver.find_element(By.CSS_SELECTOR, selector)
    value = element.text.strip()
    print("🔍 추출된 값:", value)
except Exception as e:
    print("❌ 요소를 찾을 수 없습니다:", e)

# 5. 드라이버 종료
driver.quit()


🔍 추출된 값: 4,940
(2,806)
